In [ ]:
!python -m pip install matplotlib numpy

In [ ]:
import numpy as np

def dft_matrix(n: int, inv: bool = False) -> np.ndarray:
    if not isinstance(n, int) or not isinstance(inv, bool):
        raise TypeError()
    if n < 1:
        raise ValueError()
        
    k = np.arange(n)
    w_n = np.exp((2J if inv else -2J) * k * np.pi / n)
    return np.vander(w_n, n, increasing=True) / (n if inv else 1.0)

In [ ]:
from numpy.fft import fft, ifft
from numpy.random import randint, randn

for _ in range(100):
    kwargs = {'rtol': 10E-8, 'atol': 10E-8}
    n = randint(100) + 1
    x = randn(n) + 1J * randn(n)

    y = dft_matrix(n) @ x
    assert np.allclose(y, fft(x), **kwargs)
    x = dft_matrix(n, inv=True) @ y
    assert np.allclose(x, ifft(y), **kwargs)

In [ ]:
from math import cos, pi, sin

def f(x: float) -> float:
    return 7.0 * sin(x) + 5.0 * cos(x * 7.0)

start, stop = -2.0 * pi, 2.0 * pi
x = np.linspace(start, stop, 2 ** 10 + 1)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

n = len(x)
y_noise = y + normal(loc=0.0, scale=1.0, size=n)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
# subplot.plot(np.abs(fft(y))[:len(y) // 2], color='blue', lw=2, label='fft')
type(2.0J)
import math
math.pow(3, 3)